# Testing Pipelines

## Preprocessing pipeline

In [1]:
import yaml
from pprint import pprint

from evidence_seeker import (
    PreprocessingWorkflow,
    get_openai_llm
)


config_version = "v0.1"
config_file = "../config.yaml" 

with open(config_file, 'r') as file:
    config_dict = yaml.safe_load(file)
    if config_dict['config_version'] != config_version:
        raise ValueError(f"The version of the config file does not match the used version ({config_version})")
    
#pprint(config_dict)

api_key_name = config_dict['models'][config_dict['used_model']]['api_key_name']
model = config_dict['models'][config_dict['used_model']]['model']
base_url = config_dict['models'][config_dict['used_model']]['base_url']

pw = PreprocessingWorkflow(timeout=60, verbose=False)

llm=get_openai_llm(api_key_name=api_key_name, model=model, base_url=base_url)

result = await pw.run(claim="Climate change is the biggest challenge for human kind.", 
                      llm=llm,
                      config=config_dict)
pprint(result)

Used api key name: kideku_toxicity_app_nim
Instantiating OpenAILike model (model: meta-llama/Llama-3.1-70B-Instruct, base_url: https://huggingface.co/api/integrations/dgx/v1).
Analysing ascriptive aspects of claim.
Analysing descriptive aspects of claim.
Analysing normative aspects of claim.


ValidationError: 1 validation error for NegateClaimsEvent
init_data_dict
  Input should be a valid dictionary [type=dict_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type

In [2]:
# %pip install llama-index-utils-workflow

from llama_index.utils.workflow import draw_all_possible_flows
from evidence_seeker import PreprocessingWorkflow

draw_all_possible_flows(PreprocessingWorkflow, filename="preprocessing_workflow_visualisation.html")

<class 'NoneType'>
<class 'evidence_seeker.preprocessing.ListAscriptiveClaimsEvent'>
<class 'evidence_seeker.preprocessing.ListDescriptiveClaimsEvent'>
<class 'evidence_seeker.preprocessing.AscriptiveAnalysisEndEvent'>
<class 'evidence_seeker.preprocessing.DescriptiveAnalysisEndEvent'>
<class 'evidence_seeker.preprocessing.NormativeAnalysisEndEvent'>
<class 'evidence_seeker.preprocessing.ListNormativeClaimsEvent'>
<class 'evidence_seeker.preprocessing.NormativeAnalysisEvent'>
<class 'evidence_seeker.preprocessing.DescriptiveAnalysisEvent'>
<class 'evidence_seeker.preprocessing.AscriptiveAnalysisEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
preprocessing_workflow_visualisation.html
